# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

    
#### Performance Measure: 

* Performance Measure is the cost of the optimal path found by the agent
* Maximize Cost (Prefer Travelling in the order of safe places, flooded areas and water bodies).
* Minimize Distance (Travel through less number of squares).
* Reach the goal state by Maximizing Cost and Minimizing Distance.


#### Environment: 

* 8*8 Grid-based representation of the city map with empty roads, water bodies, or flooded roads.
* The environment is fully observable.
* Cells are either empty, water bodies, or flooded roads.
* Up, down, left, and right movement allowed (not diagonals).
* The agent can travel through empty cells but not through the water bodies or flooded areas(blockades).

#### Actuators: 

* The Agents movement in the north, south, east, and west directions (up, down, left, or right).

#### Sensors: 

* The Agents vision sensors to detect empty roads, water bodies, or flooded roads.



Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)

import random
import time

POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 100

# . : Empty cell
# W : Water body
# F : Flooded area
grid = [
    ['.', '.', '.', '.', 'W', 'W', '.', '.'],
    ['.', 'F', 'F', '.', '.', '.', '.', 'F'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['W', 'W', '.', 'F', '.', '.', '.', '.'],
    ['.', 'F', '.', '.', '.', '.', 'W', 'W'],
    ['.', '.', '.', 'F', '.', '.', '.', '.'],
    ['.', 'W', 'W', '.', '.', 'F', '.', '.'],
    ['.', '.', '.', '.', '.', 'F', '.', '.']
]

In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

transitions = [(0, 1), (0, -1), (1, 0), (-1, 0)] # Right, Left, Down, Up

transition_cost = [[0 for cell in row] for row in grid] # Cost Matrix

for row in range(len(grid)):
    for cell in range(len(grid[0])):
        bonus = 0
        penalty = 0
        for dir in transitions:
            neighbor = (row + dir[0], cell + dir[1])
            if 0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0]):
                if grid[neighbor[0]][neighbor[1]] == 'F':
                    penalty += 3
                elif grid[neighbor[0]][neighbor[1]] == 'W':
                    penalty += 5
                else:
                    bonus += 5
        transition_cost[row][cell] = bonus - penalty

def heuristic(node, goal):
    x1, y1 = node
    x2, y2 = goal
    distance = (abs(x1 - x2) + abs(y1 - y2))
    return (distance) / transition_cost[x1][y1]

In [3]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

def transition_model(grid, node):
    x, y = node
    successors = []
    for dir in transitions:
        neighbor = (x + dir[0], y + dir[1])
        if 0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0]):
            if grid[neighbor[0]][neighbor[1]] != 'W' and grid[neighbor[0]][neighbor[1]] != 'F':
                successors.append(neighbor)
    return successors

In [4]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def goal_test(state, goal_state):
    return state == goal_state

### 2.	Definition of Algorithm 1 (Greedy Best-First Search)

In [5]:
#Code Block : Function for algorithm 1 implementation

# Greedy Best-First Search
def greedy_best_first_search(grid, start, goal):
    closed_list = set()
    parent = {}
    path_cost = {}
    opened_list = [(heuristic(start, goal), start)] # List to store nodes to be expanded, initialized with start node
    expanded_count = 0  
    generated_count = 0  
    max_queue_length = 0  
    while opened_list:
        max_queue_length = max(max_queue_length, len(opened_list))
        opened_list.sort(key = lambda x: x[0]) # Sorts the list based on low heuristic / fitness value
        cost, current_node = opened_list.pop(0) # Greedily select the node with lowest heuristic value
        if goal_test(current_node, goal):
            path = []
            total_cost = 0
            while current_node != start:
                path.append(current_node)
                total_cost += transition_cost[current_node[0]][current_node[1]]
                current_node = parent[current_node]
            path.append(start)
            path.reverse()
            return path, total_cost, expanded_count, generated_count, max_queue_length
        closed_list.add(current_node)
        expanded_count += 1  
        for child in transition_model(grid, current_node):
            generated_count += 1
            heuristic_child = heuristic(child, goal)
            if child not in closed_list and child not in list(map(lambda x: x[1], opened_list)):
                parent[child] = current_node
                path_cost[child] = heuristic_child
                opened_list.append((heuristic_child, child))
            elif child in list(map(lambda x: x[1], opened_list)) and parent[child] != current_node: 
            # If the child is in the opened list and its parent is different from the current node
                index = list(map(lambda x: x[1], opened_list)).index(child)
                if heuristic_child < opened_list[index][0]: # If the heuristic values of child is lower than the corresponding node in opened list
                    parent[child] = current_node
                    path_cost[child] = heuristic_child
                    opened_list.append((heuristic_child, child))
    return None, 0, expanded_count, generated_count, max_queue_length  # Path not found

### 3.	Definition of Algorithm 2 (Genetic Algorithm)

In [6]:
#Code Block : Function for algorithm 2 implementation

# Genetic Algorithm
def genetic_algorithm(POPULATION_SIZE, grid, initial_state, goal_state):
    
    # Function to generate the initial population
    def generate_population(population_size, grid, start, goal):
        population = []
        while len(population) != population_size:
            individual = [start]
            current_node = start
            while current_node != goal:
                possible_moves = []
                for possible_move in transition_model(grid, current_node):
                    if possible_move not in individual:
                        possible_moves.append(possible_move)
                if not possible_moves:
                    break
                current_node = random.choice(possible_moves)
                individual.append(current_node)
            if(goal in individual):
                population.append(individual)
        return population

    # Function to get the best path from the population
    def get_best_path(population):
        fitness_scores = []
        for individual in population:
            visited = []
            fitness_score =  0
            total_cost = 0
            for node in individual:
                fitness_score += heuristic(node, end_state)
                if node != start_state:
                    total_cost += transition_cost[node[0]][node[1]]
            fitness_scores.append((fitness_score, individual, total_cost))
            best_node = min(fitness_scores, key=lambda fitness_scores: fitness_scores[0])
        return best_node[0], best_node[1], total_cost

    # Function for crossover operation
    def crossover(parent1, parent2):
        common_points = set(parent1) & set(parent2)
        common_points = list(filter(lambda node: node!= start_state and node != end_state, list(common_points)))
        if not common_points: # If no common points, return parents without crossover
            return random.choice([parent1, parent2]) # Select a random common point for crossover
        common_point = random.choice(common_points) # Perform crossover at the selected common point
        child1 = parent1[:parent1.index(common_point)] + parent2[parent2.index(common_point):]
        child2 = parent2[:parent2.index(common_point)] + parent1[parent1.index(common_point):]
        return random.choice([child1, child2])

    # Function for mutation operation
    def mutate(path):
        random_nodes = random.sample(path, k=2)
        random_nodes.sort()
        possible_changes = generate_population(5, grid, random_nodes[0], random_nodes[1])
        mutate = random.choice(possible_changes)
        start_idx = path.index(random_nodes[0])
        end_idx = path.index(random_nodes[1])
        del path[start_idx:end_idx+1]
        for item in reversed(mutate):
            path.insert(start_idx, item)
        return path

    # Function to run a single generation of the genetic algorithm
    def run_generation(population):
        # Run a single generation of the genetic algorithm
        new_population = []
        for i in range(len(population)):
            parent1, parent2 = select_parents(population)
            child = crossover(parent1, parent2)
            if random.random() < MUTATION_RATE:
                child = mutate(child)
            new_population.append(child)
        return new_population

    # Function to select parents from the population
    def select_parents(population):
        # Select two parents from the population using tournament selection
        f_score, parent1, total_cost = get_best_path(random.sample(population, k=5))
        f_score, parent2, total_cost = get_best_path(random.sample(population, k=5))
        return parent1, parent2
    
    population = generate_population(POPULATION_SIZE, grid, initial_state, goal_state)
    f_score, best_path, total_cost = get_best_path(population)
    fitness_history = [f_score]
    generation_size = 0
    # Running the genetic algorithm for a number of generations
    for i in range(NUM_GENERATIONS):
        population = run_generation(population)
        f_score, best_path, total_cost = get_best_path(population)
        fitness_history.append(f_score)
        # Termination criteria: For last 25 generations, there is no improvement in chromosome fitness.
        if len(fitness_history) > 25 and len(set(fitness_history[-25:])) <= 1:
            break
    return fitness_history[-1], best_path, total_cost, len(fitness_history)

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [7]:
#Code Block : Function & call to get inputs (start/end state)

possible_states = []
for row in range(len(grid)):
    for cell in range(len(grid[0])):
        if(grid[row][cell] == '.'):
            possible_states.append((row,cell))

def get_state_input(state):
    choice = input(f"Please Select {state} from the Possible states: ")
    while not choice.isdigit() or int(choice) - 1 < 0 or int(choice) - 1 >= len(possible_states):
        print(f"Invalid State. Please enter a valid {state} from possible states.")
        choice = input(f"Please Select {state} from the Possible states: ")
    return possible_states[int(choice) - 1]

print("Possible States: ", end="\n\n")

for index, state in enumerate(possible_states):
    print(f"{index+1}: {state}")

start_state = get_state_input("Start State")
end_state = get_state_input("End State")

print("Start State", start_state)
print("Goal State", end_state)

Possible States: 

1: (0, 0)
2: (0, 1)
3: (0, 2)
4: (0, 3)
5: (0, 6)
6: (0, 7)
7: (1, 0)
8: (1, 3)
9: (1, 4)
10: (1, 5)
11: (1, 6)
12: (2, 0)
13: (2, 1)
14: (2, 2)
15: (2, 3)
16: (2, 4)
17: (2, 5)
18: (2, 6)
19: (2, 7)
20: (3, 2)
21: (3, 4)
22: (3, 5)
23: (3, 6)
24: (3, 7)
25: (4, 0)
26: (4, 2)
27: (4, 3)
28: (4, 4)
29: (4, 5)
30: (5, 0)
31: (5, 1)
32: (5, 2)
33: (5, 4)
34: (5, 5)
35: (5, 6)
36: (5, 7)
37: (6, 0)
38: (6, 3)
39: (6, 4)
40: (6, 6)
41: (6, 7)
42: (7, 0)
43: (7, 1)
44: (7, 2)
45: (7, 3)
46: (7, 4)
47: (7, 6)
48: (7, 7)


Please Select Start State from the Possible states:  1
Please Select End State from the Possible states:  48


Start State (0, 0)
Goal State (7, 7)


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [8]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

gbs_start_time = time.time()
path, total_cost, expanded_count, generated_count, max_queue_length = greedy_best_first_search(grid, start_state, end_state)
gbs_time_taken = time.time() - gbs_start_time

if path:
    print("Path:", path)
    print("Cost:", total_cost, end ='\n\n')
    print("Path Visualization:")
    for i in range(len(grid)):
        row = [c for c in grid[i]]
        for j in range(len(grid[0])):
            if (i,j) in path:
                row[j] = '*'
        print(row)

else:
    print("Path Not found.")

Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (5, 5), (5, 6), (6, 6), (6, 7), (7, 7)]
Cost: 172

Path Visualization:
['*', '*', '*', '*', 'W', 'W', '.', '.']
['.', 'F', 'F', '*', '.', '.', '.', 'F']
['.', '.', '.', '*', '*', '*', '.', '.']
['W', 'W', '.', 'F', '.', '*', '.', '.']
['.', 'F', '.', '.', '.', '*', 'W', 'W']
['.', '.', '.', 'F', '.', '*', '*', '.']
['.', 'W', 'W', '.', '.', 'F', '*', '*']
['.', '.', '.', '.', '.', 'F', '.', '*']


In [9]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

ga_start_time = time.time()

fitness, path, total_cost, generations_taken = genetic_algorithm(POPULATION_SIZE, grid, start_state, end_state)

ga_time_taken = time.time() - ga_start_time

if path:
    print("Path:", path)
    print("Fitness Score:", fitness)
    print("Cost:", total_cost, end ='\n\n')
    print("Path Visualization:")
    for i in range(len(grid)):
        row = [c for c in grid[i]]
        for j in range(len(grid[0])):
            if (i,j) in path:
                row[j] = '*'
        print(row)
else:
    print("No path found.")

Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (3, 4), (4, 4), (5, 4), (5, 5), (5, 6), (6, 6), (6, 7), (7, 7)]
Fitness Score: 11.321428571428573
Cost: 166

Path Visualization:
['*', '*', '*', '*', 'W', 'W', '.', '.']
['.', 'F', 'F', '*', '.', '.', '.', 'F']
['.', '.', '.', '*', '*', '.', '.', '.']
['W', 'W', '.', 'F', '*', '.', '.', '.']
['.', 'F', '.', '.', '*', '.', 'W', 'W']
['.', '.', '.', 'F', '*', '*', '*', '.']
['.', 'W', 'W', '.', '.', 'F', '*', '*']
['.', '.', '.', '.', '.', 'F', '.', '*']


### 5.	Comparitive Analysis

In [10]:
#Code Block : Print the Time & Space complexity of algorithm 1 (Greedy Best First Search Algorithm)

# Space Complexity
print("Expanded:", expanded_count)
print("Generated:", generated_count)
print("Max Queue Length:", max_queue_length)

# Time Complexity
print("Execution Time:", gbs_time_taken)

Expanded: 18
Generated: 54
Max Queue Length: 13
Execution Time: 0.0002696514129638672


In [11]:
#Code Block : Print the Time & Space complexity of algorithm 2 (Genetic Algorithm)

# Space Complexity
print("Population Size:", POPULATION_SIZE)
print("Number of Generations:", generations_taken)

# Time Complexity
print("Execution Time:", ga_time_taken)

Population Size: 50
Number of Generations: 27
Execution Time: 0.2761383056640625


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : The greedy best-first search algorithm has lower space and time complexity with fewer expansions, and shorter execution time compared to the genetic algorithm, which requires larger population and generations leading to higher execution time. The genetic algorithm's execution time is approximately 200 % higher than that of the greedy best-first search algorithm.